In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from helper_functions import *

import xgboost as xgb
from xgboost import XGBRegressor

In [2]:
result_df = pd.read_csv('compare_mape.csv')
train = pd.read_csv('train.csv')
result_df.head()

,Country,MAPE,Density,Active,forecast,category,LR_MAPE,SV_MAPE
0,1001,1.504614,3.334431,1417,[3.3091352 3.31727028 3.32801533 3.33693123 3...,LR,0.992271,1.835232
1,1003,4.143341,7.823300,13401,[7.85833788 7.88518858 7.90744925 7.90828514 7...,tf_dense,4.317589,5.230218
2,1005,1.395184,1.206827,239,[1.16141069 1.16513038 1.17019248 1.18426347 1...,SV,1.833943,1.101416
3,1007,5.228748,1.236650,220,[1.22299695 1.22010887 1.22093809 1.21361172 1...,SV,4.259990,3.574082
4,1009,1.442244,1.777708,789,[1.75237012 1.76360607 1.77065146 1.7765491 1...,LR,1.285277,2.116541


In [3]:
train.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [4]:
%%time
mapes = []
smape = []
count = 0

IDE = train.cfips.unique()
x_train = np.arange(31).reshape((-1,1))
x_test = np.arange(31,39).reshape((-1,1))
# Iterate all the cfips
for i in range(len(IDE)):
    c = result_df['Country'].iloc[i]
    df = train.loc[train.cfips == c]
    last = df.microbusiness_density.values[-9]
    
    xgbr = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
    
    #xgbr = XGBRegressor(objective='reg:pseudohubererror', 
     #                  n_estimators=4999, 
      #                  learning_rate=0.0075,
       #                 max_leaves = 17, 
        #                subsample=0.50,
         #               colsample_bytree=0.50, 
          #              max_bin=4096,
           #             n_jobs=2,
            #            eval_metric='mae',
             #          )
    
    xgbr.fit(x_train, df['microbusiness_density'][:31])
    preds = xgbr.predict(x_test)
    
    y_test = np.expand_dims(np.array(df['microbusiness_density'][31:]), axis=0)
    results = evaluate_preds(y_test, preds)
    m = results['mape']
    sm = results['smape']
    smape.append(sm)
    mapes.append(m)
    
    count+=1
    print(count)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [10]:
y_true = train[train['cfips'] == 1001]['microbusiness_density'][-8:]
y_pred = np.array([3.3256, 3.41046, 3.31567, 3.4265, 3.41632, 3.3678, 3.64321, 3.512345])

smap = np.zeros(len(y_true))
num = np.abs(y_true - y_pred)
print(num)
dem = ((np.abs(y_true) + np.abs(y_true)) / 2)
print(dem)
pos_ind = (y_true!=0)|(y_pred!=0)
print(pos_ind)
smap[pos_ind] = num[pos_ind] / dem[pos_ind]
smap

31    0.011185
32    0.038378
33    0.002417
34    0.080303
35    0.021651
36    0.056052
37    0.200533
38    0.048489
Name: microbusiness_density, dtype: float64
31    3.336785
32    3.372082
33    3.313253
34    3.346197
35    3.437971
36    3.423852
37    3.442677
38    3.463856
Name: microbusiness_density, dtype: float64
31    True
32    True
33    True
34    True
35    True
36    True
37    True
38    True
Name: microbusiness_density, dtype: bool


array([0.00335191, 0.0113811 , 0.00072952, 0.02399817, 0.00629749,
       0.01637095, 0.05824915, 0.01399871])

In [15]:
import tensorflow as tf
tf.cast(y_true, dtype=tf.float32).numpy()

array([3.3367846, 3.372082 , 3.313253 , 3.3461974, 3.4379706, 3.4238517,
       3.442677 , 3.4638555], dtype=float32)

In [43]:
result_df['XGBR_MAPE'] = mapes
mape_table = result_df[['Country', 'MAPE', 'LR_MAPE', 'SV_MAPE', 'XGBR_MAPE']]
mape_table = mape_table.rename(columns={'MAPE': "tf_dense_MAPE"})
mape_table

,Country,tf_dense_MAPE,LR_MAPE,SV_MAPE,XGBR_MAPE
0,1001,1.504614,0.992271,1.835232,1.983559
1,1003,4.143341,4.317589,5.230218,5.321415
2,1005,1.395184,1.833943,1.101416,1.044702
3,1007,5.228748,4.259990,3.574082,3.717209
4,1009,1.442244,1.285277,2.116541,2.159934
...,...,...,...,...,...
3130,56037,8.937141,3.550418,2.450784,2.473019
3131,56039,3.437833,2.390681,1.586990,1.590857
3132,56041,4.339747,3.925458,1.381605,1.527132
3133,56043,2.373714,2.219479,3.608545,3.606808


In [44]:
best_scores = mape_table.min(axis='columns')
cat = []
for i, m in enumerate(best_scores):
    if m == mape_table['tf_dense_MAPE'].iloc[i]:
        category = 'tf_dense'
        cat.append(category)
    elif m == mape_table["LR_MAPE"].iloc[i]:
        category = "LR"
        cat.append(category)
    elif m == mape_table['SV_MAPE'].iloc[i]:
        category = "SV"
        cat.append(category)
    else:
        category = 'XGBR'
        cat.append(category)
mape_table['category'] = cat
mape_table.head()

,Country,tf_dense_MAPE,LR_MAPE,SV_MAPE,XGBR_MAPE,category
0,1001,1.504614,0.992271,1.835232,1.983559,LR
1,1003,4.143341,4.317589,5.230218,5.321415,tf_dense
2,1005,1.395184,1.833943,1.101416,1.044702,XGBR
3,1007,5.228748,4.259990,3.574082,3.717209,SV
4,1009,1.442244,1.285277,2.116541,2.159934,LR


In [45]:
mape_table['category'].value_counts()

SV          1051
LR           845
tf_dense     793
XGBR         446
Name: category, dtype: int64

In [46]:
mape_table.describe()

,Country,tf_dense_MAPE,LR_MAPE,SV_MAPE,XGBR_MAPE
count,3135.000000,3135.000000,3.135000e+03,3135.000000,3135.000000
mean,30376.037640,27.996707,8.927551e+03,3.909992,75.852959
std,15145.862593,1269.835687,4.995858e+05,21.621562,4026.084473
min,1001.000000,0.175896,2.424637e-01,0.000000,0.009689
25%,18178.000000,1.800255,1.664201e+00,1.394263,1.419737
50%,29173.000000,3.101383,2.748950e+00,2.353172,2.394855
75%,45076.000000,5.295601,4.815689e+00,3.993313,4.054720
max,56045.000000,71091.580000,2.797235e+07,1183.264800,225429.937500


In [41]:
mape_table.to_csv('add_XGBR_results.csv', index=False)

In [48]:
mape_table.to_csv('add_XGBR_result_2.csv', index=False)

In [49]:
d = pd.read_csv('add_XGBR_results.csv')
d.describe()

,Country,tf_dense_MAPE,LR_MAPE,SV_MAPE,XGBR_MAPE
count,3135.000000,3135.000000,3.135000e+03,3135.000000,3135.000000
mean,30376.037640,27.996707,8.927551e+03,3.909992,116.271379
std,15145.862593,1269.835687,4.995858e+05,21.621562,4497.496999
min,1001.000000,0.175896,2.424637e-01,0.000000,0.020653
25%,18178.000000,1.800255,1.664201e+00,1.394263,2.300055
50%,29173.000000,3.101383,2.748950e+00,2.353172,5.577039
75%,45076.000000,5.295601,4.815689e+00,3.993313,89.062245
max,56045.000000,71091.580000,2.797235e+07,1183.264800,251844.530000
